In [13]:
# 🔐 Step 1: Google Drive 認証
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.client import GoogleCredentials

import pandas as pd
import datetime
import pytz

creds = GoogleCredentials.get_application_default()
drive_service = build('drive', 'v3', credentials=creds)

In [14]:
# 📁 Step 2: Drive ファイル一覧の取得
def list_drive_files():
    results = drive_service.files().list(
        pageSize=1000,
        fields="files(id, name, size, mimeType, modifiedTime, parents)"
    ).execute()
    files = results.get('files', [])
    return pd.DataFrame(files)

df = list_drive_files()
df['size'] = df['size'].astype(float) / (1024 * 1024)  # MB変換
df['modifiedTime'] = pd.to_datetime(df['modifiedTime'])

# サイズや古さによる分類
utc_now = datetime.datetime.now(pytz.utc)
old_files = df[df['modifiedTime'] < (utc_now - datetime.timedelta(days=365))]
large_files = df[df['size'] > 50]

In [15]:
# 📁 Step 3: アーカイブフォルダの作成・取得
def get_or_create_folder(name, parent_id=None):
    query = f"name='{name}' and mimeType='application/vnd.google-apps.folder'"
    if parent_id:
        query += f" and '{parent_id}' in parents"
    results = drive_service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
    folders = results.get('files', [])
    if folders:
        return folders[0]['id']
    else:
        folder_metadata = {
            'name': name,
            'mimeType': 'application/vnd.google-apps.folder',
        }
        if parent_id:
            folder_metadata['parents'] = [parent_id]
        folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
        return folder.get('id')

# フォルダ階層：Archive > Videos > FLOW-3D
archive_id = get_or_create_folder("Archive")
videos_id = get_or_create_folder("Videos", parent_id=archive_id)
flow3d_id = get_or_create_folder("FLOW-3D", parent_id=videos_id)
print(f"Target folder ID: {flow3d_id}")

Target folder ID: 17Q4id9RG6uKj7jTTG-vkd1Tqt1mxBUDu


In [16]:
# 📦 Step 4: 対象ファイル名のリスト
file_names_to_move = [
    "HPDC- Filling (Part 2).mp4",
    "What's New in FLOW-3D CAST v5.mp4",
    "What's New in FLOW-3D Cast v4.1.mp4",
    "What's New in FLOW-3D Cast v4.2.mp4",
    "HPDC Thermal Die Cycling.mp4",
    "HPDC Solidification.mp4",
    "Simplified Filling - FLOW-3D Users Site.mp4"
]

In [17]:
# 📤 Step 5: ファイル移動用の関数（アポストロフィ対応付き）
def escape_drive_query(s):
    return s.replace("'", r"\'")  # クエリ用にエスケープ

def move_files_to_folder(file_names, destination_folder_id):
    for name in file_names:
        escaped_name = escape_drive_query(name)
        query = f"name='{escaped_name}' and trashed=false"
        try:
            results = drive_service.files().list(
                q=query,
                fields="files(id, name, parents)",
                spaces='drive'
            ).execute()
            files = results.get('files', [])
            if not files:
                print(f"❌ Not found: {name}")
                continue
            file = files[0]
            file_id = file['id']
            prev_parents = ",".join(file.get('parents', []))
            drive_service.files().update(
                fileId=file_id,
                addParents=destination_folder_id,
                removeParents=prev_parents,
                fields='id, parents'
            ).execute()
            print(f"✅ Moved: {name}")
        except Exception as e:
            print(f"❌ Failed: {name} → {e}")

In [18]:
# 🚀 Step 6: ファイル移動の実行
move_files_to_folder(file_names_to_move, flow3d_id)

✅ Moved: HPDC- Filling (Part 2).mp4
✅ Moved: What's New in FLOW-3D CAST v5.mp4
✅ Moved: What's New in FLOW-3D Cast v4.1.mp4
✅ Moved: What's New in FLOW-3D Cast v4.2.mp4
✅ Moved: HPDC Thermal Die Cycling.mp4
✅ Moved: HPDC Solidification.mp4
✅ Moved: Simplified Filling - FLOW-3D Users Site.mp4
